In [1]:
# Connet mssql to retrieve hotel data

import pymssql #引入pymssql模块
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import re

def conn():
    connect = pymssql.connect('121.42.61.195', 'test', 'tes@t123456#pwd2018&', 'Broker') #服务器名,账户,密码,数据库名
    if connect:
        print("连接成功!")
    return connect

conn = conn()

cursor = conn.cursor()   #创建一个游标对象,python里的sql语句都要通过cursor来执行
sql = "select planner_id, wedding_id, province, city, hotel, hotel_id from dbo.tb_hotel_wedding_planner_data"

df0 = pd.read_sql(sql,conn)
df=pd.DataFrame(df0)

print (df.head())

cursor.close()   
conn.close()

连接成功!
                             planner_id                            wedding_id  \
0  fe8c2460-0971-4973-8a34-e42019e253d8  c20a17ed-98dc-45fb-b522-65d77753bc78   
1  fe8c2460-0971-4973-8a34-e42019e253d8  ca22e2ea-5963-4f14-887a-521ff372f0f0   
2  fe8c2460-0971-4973-8a34-e42019e253d8  78bf6ad4-dd9f-4a75-a4c9-191ad8f590fd   
3  fe8c2460-0971-4973-8a34-e42019e253d8  20b520d9-c55c-4bb4-ba41-7ef4fb180136   
4  fe8c2460-0971-4973-8a34-e42019e253d8  2bc66a9f-2d66-4b31-bb2a-d0301d1db85c   

  province city      hotel                              hotel_id  
0      重庆市  重庆市  澳维酒店(西南门)  2632b834-759d-4e1b-a8d9-8ef4d7a40aec  
1      重庆市  重庆市   重庆凯宾斯基酒店  171b36dc-df60-4a0e-af60-34fb81114819  
2      重庆市  重庆市     佳宇英皇酒店  f05449d7-2a6c-4614-b77e-5c7fe8d5226f  
3      重庆市  重庆市   双山国际花园酒店  c25e0f1c-4fff-4ec6-bbe1-a2ecb558fd40  
4      重庆市  重庆市     银鑫世纪酒店  9ed4e2ad-857f-444b-8f32-75aaf4ecb882  


In [2]:
# explore data, and then merge hotel name with province/city name into a new column
print ('unique wedding id: ', df.wedding_id.nunique(dropna=False))
print ('Null wedding id: ', df.wedding_id.isnull().sum())
print ('Non_Null wedding id: ', df.wedding_id.count()) # 非空元素个数
print ('dup wedding id: ', df.wedding_id.duplicated().sum())
#print ('dup wedding id: ', df.wedding_id.value_counts())
print (df.shape)

#df['merged'] = df['province'].map(str) + ' '+ df['city'].map(str)+ ' ' +df['hotel'].map(str)
df['merged'] = df['city'].map(str)+ ' ' +df['hotel'].map(str)
df['merged']  = df['merged'].map(lambda x: x.replace('None', ''))
df['merged']  = df['merged'].map(lambda x: x.strip())


line = "四川省  四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"
#line = "重庆市 重庆市 澳维酒店(西南门)"


# input: "四川省  四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"
# output: "四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"

def removeDupProvinceAndCity(line):
    if line:      
        # remove dup province
        matchObj = re.search( r'(\b.+省\b)', line)
        if matchObj and len(matchObj.groups()) > 0:

            #print ("matchObj.group() :", matchObj)
            print ("province match, matchObj.group(1) : ", matchObj.group(1))
            result = line.find(matchObj.group(1), len(matchObj.group(1)))
            if result != -1:
                line = line[len(matchObj.group(1)):].strip()
        else:
            pass
            #print ("no match!")
            
        # remove dup city    
        matchObj = re.search( r'(\b\w+?市\b)', line)
        if matchObj and len(matchObj.groups()) > 0:

            #print ("matchObj.group() :", matchObj)
            print ("city match, matchObj.group(1) : ", matchObj.group(1))
            result = line.find(matchObj.group(1), len(matchObj.group(1)))
            if result != -1:
                line = line[len(matchObj.group(1)):].strip()
        else:
            pass
            #print ("no match!")
    return line

#removeDupProvinceAndCity(line)
#print ("result ", line)

print (df['merged'].head(10))

df['merged']  = df['merged'].apply(lambda x: removeDupProvinceAndCity(x))
    
print (df['merged'].head(10))



unique wedding id:  17254
Null wedding id:  0
Non_Null wedding id:  17300
dup wedding id:  46
(17300, 6)
0        重庆市 澳维酒店(西南门)
1         重庆市 重庆凯宾斯基酒店
2           重庆市 佳宇英皇酒店
3         重庆市 双山国际花园酒店
4           重庆市 银鑫世纪酒店
5             重庆市 渝州宾馆
6      重庆市 重庆典雅华美达广场酒店
7    重庆市 俏巴渝(爱琴海购物公园店)
8             重庆市 海琴酒店
9    重庆市 维也纳国际酒店(观音桥店)
Name: merged, dtype: object
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city mat

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

city match, matchObj.group(1) :  绵阳市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  南充市
city match, matchObj.group(1) :  德阳市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  乐山市
city match, matchObj.group(1) :  眉山市
city match, matchObj.group(1) :  乐山市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  德阳市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  德阳市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  达州市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
c

city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  眉山市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  达州市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  眉山市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  眉山市
c

city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  四川省成都市都江堰市都江堰市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  眉山市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(

city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  雅安市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  眉山市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  德阳市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
city match, matchObj.group(1) :  成都市
c

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

In [3]:
import jieba

def word_segmentation(line):
    if line:
        #print (line)
        cut_words = list(jieba.cut(line))
        line = " ".join(cut_words)

    return line


df['merged_cut']  = df['merged'].apply(lambda x: word_segmentation(x))
   
print (df['merged_cut'].tail())
print (df.head())    


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.067 seconds.
Prefix dict has been built succesfully.


17295         鼎 宏大 酒楼 合阳 大道 654 号 附 13 号 宝龙 城市 广场 3 层
17296                                            福寿 镇
17297    友缘 山庄 合阳 城南 津街 街道 花园村 龙眼 井 村民 小组 ( 铜梁 洞 后山 )
17298                          重庆 宝龙 戴斯 大酒店 濮岩路 223 号
17299                          希尔 安江 上城 希尔 安 大道 168 号
Name: merged_cut, dtype: object
                             planner_id                            wedding_id  \
0  fe8c2460-0971-4973-8a34-e42019e253d8  c20a17ed-98dc-45fb-b522-65d77753bc78   
1  fe8c2460-0971-4973-8a34-e42019e253d8  ca22e2ea-5963-4f14-887a-521ff372f0f0   
2  fe8c2460-0971-4973-8a34-e42019e253d8  78bf6ad4-dd9f-4a75-a4c9-191ad8f590fd   
3  fe8c2460-0971-4973-8a34-e42019e253d8  20b520d9-c55c-4bb4-ba41-7ef4fb180136   
4  fe8c2460-0971-4973-8a34-e42019e253d8  2bc66a9f-2d66-4b31-bb2a-d0301d1db85c   

  province city      hotel                              hotel_id  \
0      重庆市  重庆市  澳维酒店(西南门)  2632b834-759d-4e1b-a8d9-8ef4d7a40aec   
1      重庆市  重庆市   重庆凯宾斯基酒店  171b36dc-df60-4a0e-af60-34fb81114819   
2      

In [4]:
def calcTfidfMatrix(document):
    tfidf_model = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b").fit(document)
    #print(tfidf_model.vocabulary_)
    tfidf = tfidf_model.transform(document)

    simi_matrix = (tfidf * tfidf.T).A #similarities matrix of sentences
    #simi_matrix = np.tril(simi_matrix) # only use the lower left part of the matrix
    return simi_matrix


In [22]:
# encoding=utf8  
import numpy as np

hotel_indicator = [u'酒店', u'酒楼', u'饭店', u'宾馆', u'会所', u'会馆', u'食府', u'庄园', u'人家']

def is_in_dict_values(dict, value):
    if not dict:
        return False
    for key, values in dict.items():
        if value in values:
            return True
    return False
    

def save2excel(pd_simi_matrix):
    pd_simi_matrix_g = pd_simi_matrix # full filled matrix
    pd_simi_matrix = pd.DataFrame(np.tril(pd_simi_matrix.values), columns=pd_simi_matrix.index.values.tolist(), index=pd_simi_matrix.index.values.tolist()) # only use the lower left part of the matrix

    col_names = pd_simi_matrix.columns.values.tolist()
    row_names = pd_simi_matrix.index.values.tolist() 
    threshold_low = 0.7
    threshold_high = 0.75

    pd_excel = None
    if os.path.exists("result.csv"):
        pd_excel = pd.read_csv("result.csv", index_col='ID')
        
    pd_simi_matrix_tmp = pd_simi_matrix.where(pd_simi_matrix > threshold_low, 0)
    
    simi_dict = {}

    for col in col_names:
        if not is_in_dict_values(simi_dict, col):
            
            rows_with_large_similarity = [row for row in row_names if pd_simi_matrix_tmp.loc[row, col] > threshold_low]         
            print ("col:", col, "similar rows: ", rows_with_large_similarity)

            data = None
            possible_incomplete_hotelnames2 = []
            possible_incomplete_hotelnames3 = []
            
            if len(rows_with_large_similarity) == 0: # for any hotel name with no charector
                print ("00000000000000000000")
                rows_with_large_similarity.append(col)
                data = pd.DataFrame(df, columns=['planner_id', 'wedding_id', 'hotel', 'hotel_id', 'merged'], index=rows_with_large_similarity)
                possible_incomplete_hotelnames2.append(col)
                print (data.head())
            elif (len(rows_with_large_similarity) == 1): # only one item for the hotel name
                print ("11111111111111111111")
                data = pd.DataFrame(df, columns=['planner_id', 'wedding_id', 'hotel', 'hotel_id', 'merged'], index=rows_with_large_similarity)
                if len(data.loc[col, 'hotel']) <= 4:
                    possible_incomplete_hotelnames2.append(col)
            else:
                simi_dict[col] = rows_with_large_similarity
            
                data = pd.DataFrame(df, columns=['planner_id', 'wedding_id', 'hotel', 'hotel_id', 'merged'], index=rows_with_large_similarity)

                # TODO: 如果名字是 会展，南平会展，这里推荐的是会展，应该改成推荐长名字
                temp_rec_name = data['hotel'].value_counts().idxmax()
                data['recommended_name'] = temp_rec_name
                temp_data = data[data['hotel'] == temp_rec_name]
                idx_most_possible = temp_data.index.tolist()[0]

                # find similarity around idx_most_possible from similarity matrix
                others_simi = [i for i in rows_with_large_similarity if i != idx_most_possible]
                temp_series = pd_simi_matrix_g.loc[others_simi, idx_most_possible]
                if col == 4521:
                    print (temp_series.head(20))

                possible_incomplete_hotelnames = temp_series[temp_series <= threshold_low].index.tolist()
                possible_incomplete_hotelnames2 = [row for row in possible_incomplete_hotelnames
                                                   if (len(data.loc[row, 'hotel']) <= 4 and len(data.loc[row, 'hotel']) < len(temp_rec_name))]


            for row in possible_incomplete_hotelnames2:
                print ("hotelname2", possible_incomplete_hotelnames2)
                if len(data.loc[row, 'hotel']) <= 2:
                    possible_incomplete_hotelnames3.append(row)
                    print ("wrong name!!!!")
                    continue
                    
                tmp_list = [i for i in hotel_indicator if data.loc[row, 'hotel'].find(i)>=0]
                if not any(tmp_list):
                    possible_incomplete_hotelnames3.append(row)
                               
            print ("incomplete name 3, col:", col, "row:", possible_incomplete_hotelnames3)
            
            data['possible_incomplete_hotelnames']= ""

            # if similarity is lower than others, and length is shorter than average, we can assume that's a abnormal hotel name
            data.loc[possible_incomplete_hotelnames3, 'possible_incomplete_hotelnames'] = "yes"

            if pd_excel is None:
                pd_excel = data
            else:
                pd_excel = pd.concat([pd_excel, data])
    
    # de-dup items with same index
    #idx = np.unique(pd_excel.index, return_index=True)[1]
    #pd_excel = pd_excel.iloc[idx]
    
    pd_excel['index_col'] = pd_excel.index
    pd_excel= pd_excel.drop_duplicates(subset=['index_col'], keep='first')
    
    pd_excel.to_csv("result.csv", index_label="ID")
    
    print ("end")

In [23]:
# split the big df to several small ones according to 'Province' columns

uni_values = df.province.unique()
print (uni_values) 
print ('Null  id: ', df.province.isnull().sum())
small_dfs = [df[df.province==i] for i in uni_values if i is not None]
print (len(small_dfs))
print (small_dfs[0].head())

batch_size = 3000

for sml_df in small_dfs:
    steps = sml_df.shape[0]//batch_size
    for step in range(steps+1):
        if step == steps:
            batch_df = sml_df.iloc[step * batch_size:sml_df.shape[0]]
        else:
            batch_df = sml_df.iloc[step * batch_size:(step+1) * batch_size]
        
        simi_matrix = calcTfidfMatrix(batch_df['merged_cut'].tolist())
        pd_simi_matrix = pd.DataFrame(simi_matrix, columns=batch_df.index.values.tolist(), index=batch_df.index.values.tolist())

        save2excel(pd_simi_matrix)



['重庆市' '贵州省' '海南省' '江西省' '四川省' '云南省' None]
Null  id:  67
6
                             planner_id                            wedding_id  \
0  fe8c2460-0971-4973-8a34-e42019e253d8  c20a17ed-98dc-45fb-b522-65d77753bc78   
1  fe8c2460-0971-4973-8a34-e42019e253d8  ca22e2ea-5963-4f14-887a-521ff372f0f0   
2  fe8c2460-0971-4973-8a34-e42019e253d8  78bf6ad4-dd9f-4a75-a4c9-191ad8f590fd   
3  fe8c2460-0971-4973-8a34-e42019e253d8  20b520d9-c55c-4bb4-ba41-7ef4fb180136   
4  fe8c2460-0971-4973-8a34-e42019e253d8  2bc66a9f-2d66-4b31-bb2a-d0301d1db85c   

  province city      hotel                              hotel_id  \
0      重庆市  重庆市  澳维酒店(西南门)  2632b834-759d-4e1b-a8d9-8ef4d7a40aec   
1      重庆市  重庆市   重庆凯宾斯基酒店  171b36dc-df60-4a0e-af60-34fb81114819   
2      重庆市  重庆市     佳宇英皇酒店  f05449d7-2a6c-4614-b77e-5c7fe8d5226f   
3      重庆市  重庆市   双山国际花园酒店  c25e0f1c-4fff-4ec6-bbe1-a2ecb558fd40   
4      重庆市  重庆市     银鑫世纪酒店  9ed4e2ad-857f-444b-8f32-75aaf4ecb882   

          merged            merged_cut  
0  重

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:96: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



col: 12 similar rows:  [12, 993, 1106, 1792, 1798, 2228]
incomplete name 3, col: 12 row: []
col: 13 similar rows:  [13, 27]
incomplete name 3, col: 13 row: []
col: 14 similar rows:  [14, 313, 624, 782, 2371, 2373, 2384, 2492]
incomplete name 3, col: 14 row: []
col: 15 similar rows:  [15, 2288, 2299]
incomplete name 3, col: 15 row: []
col: 16 similar rows:  [16, 38, 145, 393, 635, 637, 642, 673, 1263, 1489, 1856, 1928, 2101, 2400, 13414, 13584]
incomplete name 3, col: 16 row: []
col: 17 similar rows:  [17, 1794, 2602, 13546]
incomplete name 3, col: 17 row: []
col: 18 similar rows:  [18, 118, 266, 302, 706, 933, 1633, 1702, 1774, 1919, 1963, 1981, 2070, 2092, 2215, 2604, 13430, 13532]
incomplete name 3, col: 18 row: []
col: 19 similar rows:  [19, 309, 443, 13474, 13481]
incomplete name 3, col: 19 row: []
col: 21 similar rows:  [21, 297, 427, 463, 595, 743, 865, 987, 1659, 1770, 1786, 2134, 2320, 2523, 13530]
incomplete name 3, col: 21 row: []
col: 22 similar rows:  [22, 413, 664, 1716, 1

col: 130 similar rows:  [130, 283, 432, 643, 650, 889, 1029, 1505, 1636, 1898, 1987, 2080, 2102, 2127, 2132, 2558, 13680]
incomplete name 3, col: 130 row: []
col: 131 similar rows:  [131]
11111111111111111111
incomplete name 3, col: 131 row: []
col: 133 similar rows:  [133, 688, 927]
incomplete name 3, col: 133 row: []
col: 134 similar rows:  [134]
11111111111111111111
incomplete name 3, col: 134 row: []
col: 135 similar rows:  [135, 471, 591, 783, 965, 973, 1031, 1065, 1129, 1467, 1518, 1641, 1708, 2034, 2035, 2608, 13602]
incomplete name 3, col: 135 row: []
col: 136 similar rows:  [136]
11111111111111111111
incomplete name 3, col: 136 row: []
col: 138 similar rows:  [138, 817, 905, 1787, 2019, 13551]
incomplete name 3, col: 138 row: []
col: 139 similar rows:  [139, 1142]
incomplete name 3, col: 139 row: []
col: 140 similar rows:  [140, 144, 827, 13385]
incomplete name 3, col: 140 row: []
col: 141 similar rows:  [141, 464, 1929, 13564]
incomplete name 3, col: 141 row: []
col: 143 simi

col: 259 similar rows:  [259]
11111111111111111111
incomplete name 3, col: 259 row: []
col: 261 similar rows:  [261, 1591]
incomplete name 3, col: 261 row: []
col: 262 similar rows:  [262, 733]
incomplete name 3, col: 262 row: []
col: 263 similar rows:  [263, 738, 1887, 1943, 2591]
incomplete name 3, col: 263 row: []
col: 264 similar rows:  [264]
11111111111111111111
hotelname2 [264]
incomplete name 3, col: 264 row: []
col: 265 similar rows:  [265, 1882, 13531]
incomplete name 3, col: 265 row: []
col: 267 similar rows:  [267]
11111111111111111111
incomplete name 3, col: 267 row: []
col: 268 similar rows:  [268]
11111111111111111111
incomplete name 3, col: 268 row: []
col: 272 similar rows:  [272, 638, 1896]
incomplete name 3, col: 272 row: []
col: 273 similar rows:  [273, 424]
incomplete name 3, col: 273 row: []
col: 274 similar rows:  [274, 988]
incomplete name 3, col: 274 row: []
col: 276 similar rows:  [276, 351, 417, 573, 1876, 2287, 2313, 2318, 2390, 2561, 2631]
incomplete name 3,

...

col: 16421 similar rows:  [16421, 16631, 17159]
incomplete name 3, col: 16421 row: []
col: 16426 similar rows:  [16426]
11111111111111111111
hotelname2 [16426]
incomplete name 3, col: 16426 row: [16426]
col: 16427 similar rows:  [16427, 16458]
incomplete name 3, col: 16427 row: []
col: 16429 similar rows:  [16429, 16519]
incomplete name 3, col: 16429 row: []
col: 16430 similar rows:  [16430]
11111111111111111111
incomplete name 3, col: 16430 row: []
col: 16431 similar rows:  [16431]
11111111111111111111
hotelname2 [16431]
incomplete name 3, col: 16431 row: [16431]
col: 16433 similar rows:  [16433, 16597]
incomplete name 3, col: 16433 row: []
col: 16434 similar rows:  [16434]
11111111111111111111
incomplete name 3, col: 16434 row: []
col: 16439 similar rows:  [16439]
11111111111111111111
incomplete name 3, col: 16439 row: []
col: 16441 similar rows:  [16441]
11111111111111111111
incomplete name 3, col: 16441 row: []
col: 16442 similar rows:  [16442]
11111111111111111111
incomplete name 

col: 16590 similar rows:  [16590]
11111111111111111111
incomplete name 3, col: 16590 row: []
col: 16591 similar rows:  [16591]
11111111111111111111
incomplete name 3, col: 16591 row: []
col: 16593 similar rows:  [16593]
11111111111111111111
incomplete name 3, col: 16593 row: []
col: 16594 similar rows:  [16594]
11111111111111111111
incomplete name 3, col: 16594 row: []
col: 16595 similar rows:  [16595]
11111111111111111111
incomplete name 3, col: 16595 row: []
col: 16596 similar rows:  [16596]
11111111111111111111
hotelname2 [16596]
wrong name!!!!
incomplete name 3, col: 16596 row: [16596]
col: 16598 similar rows:  [16598]
11111111111111111111
incomplete name 3, col: 16598 row: []
col: 16599 similar rows:  [16599]
11111111111111111111
hotelname2 [16599]
incomplete name 3, col: 16599 row: [16599]
col: 16600 similar rows:  [16600]
11111111111111111111
incomplete name 3, col: 16600 row: []
col: 16603 similar rows:  [16603]
11111111111111111111
incomplete name 3, col: 16603 row: []
col: 16

col: 16762 similar rows:  [16762]
11111111111111111111
incomplete name 3, col: 16762 row: []
col: 16763 similar rows:  [16763]
11111111111111111111
incomplete name 3, col: 16763 row: []
col: 16764 similar rows:  [16764]
11111111111111111111
incomplete name 3, col: 16764 row: []
col: 16765 similar rows:  [16765]
11111111111111111111
incomplete name 3, col: 16765 row: []
col: 16766 similar rows:  [16766]
11111111111111111111
incomplete name 3, col: 16766 row: []
col: 16767 similar rows:  [16767]
11111111111111111111
incomplete name 3, col: 16767 row: []
col: 16768 similar rows:  [16768]
11111111111111111111
incomplete name 3, col: 16768 row: []
col: 16769 similar rows:  [16769]
11111111111111111111
incomplete name 3, col: 16769 row: []
col: 16770 similar rows:  [16770]
11111111111111111111
incomplete name 3, col: 16770 row: []
col: 16771 similar rows:  [16771]
11111111111111111111
incomplete name 3, col: 16771 row: []
col: 16773 similar rows:  [16773]
11111111111111111111
incomplete name

col: 16916 similar rows:  [16916]
11111111111111111111
incomplete name 3, col: 16916 row: []
col: 16918 similar rows:  [16918]
11111111111111111111
incomplete name 3, col: 16918 row: []
col: 16920 similar rows:  [16920]
11111111111111111111
incomplete name 3, col: 16920 row: []
col: 16923 similar rows:  [16923]
11111111111111111111
incomplete name 3, col: 16923 row: []
col: 16925 similar rows:  [16925]
11111111111111111111
incomplete name 3, col: 16925 row: []
col: 16930 similar rows:  [16930]
11111111111111111111
incomplete name 3, col: 16930 row: []
col: 16931 similar rows:  [16931]
11111111111111111111
incomplete name 3, col: 16931 row: []
col: 16940 similar rows:  [16940]
11111111111111111111
incomplete name 3, col: 16940 row: []
col: 16941 similar rows:  [16941]
11111111111111111111
incomplete name 3, col: 16941 row: []
col: 16944 similar rows:  [16944]
11111111111111111111
incomplete name 3, col: 16944 row: []
col: 16945 similar rows:  [16945]
11111111111111111111
incomplete name

col: 17083 similar rows:  [17083]
11111111111111111111
incomplete name 3, col: 17083 row: []
col: 17086 similar rows:  [17086]
11111111111111111111
incomplete name 3, col: 17086 row: []
col: 17087 similar rows:  [17087]
11111111111111111111
incomplete name 3, col: 17087 row: []
col: 17089 similar rows:  [17089]
11111111111111111111
hotelname2 [17089]
incomplete name 3, col: 17089 row: [17089]
col: 17091 similar rows:  [17091]
11111111111111111111
incomplete name 3, col: 17091 row: []
col: 17094 similar rows:  [17094]
11111111111111111111
hotelname2 [17094]
incomplete name 3, col: 17094 row: [17094]
col: 17095 similar rows:  [17095, 17116]
incomplete name 3, col: 17095 row: []
col: 17098 similar rows:  [17098, 17140, 17148]
incomplete name 3, col: 17098 row: []
col: 17099 similar rows:  [17099]
11111111111111111111
hotelname2 [17099]
incomplete name 3, col: 17099 row: [17099]
col: 17100 similar rows:  [17100]
11111111111111111111
incomplete name 3, col: 17100 row: []
col: 17105 similar 